In [2]:
from datetime import datetime, timezone
from facebook_scraper import get_posts, set_cookies, exceptions
import time
import pandas as pd
import json

In [17]:
set_cookies("cookies.json")

In [4]:
def handle_pagination_url(url):
    global start_url
    
    start_url = url

In [12]:
start_url = None
i=0

In [15]:
start_url

'https://m.facebook.com/groups/373920943948661?bac=MTYzNjExODU0Mzo0NjIxNzM5OTE3OTAwMjI6NDYyMTczOTkxNzkwMDIyLDAsMjc6MjA6S3c9PQ%3D%3D&multi_permalinks'

In [13]:
#              long haulers support, covid19 the long-haulers
group_ids = ["373920943948661","365867864454134"]

In [16]:
i

654

In [18]:
results = []
now = datetime.now()
nowtime = "{}-{}-{}_{}-{}-{}".format(now.year,now.month,now.day,now.hour,now.minute,now.second)
while True:
    try :
        for post in get_posts(group = group_ids[0], page_limit = None, pages = 1000, start_url = start_url, encoding="utf-8",
        request_url_callback = handle_pagination_url, options={"posts_per_page":1000,"allow_extra_requests":True,"comments":True}):
            i+=1
            if (i%100)==0:
                now = datetime.now()
                nowtime = "{}-{}-{}_{}-{}-{}".format(now.year,now.month,now.day,now.hour,now.minute,now.second)
                print("{} ## {} posts processed".format(nowtime, i))
            results.append(post)
        all_posts = pd.DataFrame()
        for each in results:
            all_posts = all_posts.append(each, ignore_index = True)
        if len(all_posts)>0:
            all_posts.to_csv("fb_scraper_{}.csv".format(nowtime))
            print("{} ## File written with timestamp".format(nowtime))
        print("Finished")
        break
    except exceptions.TemporarilyBanned:
        now = datetime.now()
        nowtime = "{}-{}-{}_{}-{}-{}".format(now.year,now.month,now.day,now.hour,now.minute,now.second)
        print("{} ## TEMPORARY BAN at {} posts... sleeping for 1 hour".format(nowtime,i))
        all_posts = pd.DataFrame()
        for each in results:
            all_posts = all_posts.append(each, ignore_index = True)
        if len(all_posts)>0:
            all_posts.to_csv("fb_scraper_{}.csv".format(nowtime))
            print("{} ## File written with timestamp".format(nowtime))
        results = []
        time.sleep(3600)


C:\Users\keatu\Anaconda3\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


2022-2-7_22-2-17 ## 700 posts processed


In [13]:
all_posts = pd.DataFrame()
for each in results:
    all_posts = all_posts.append(each, ignore_index = True)
if len(all_posts)>0:
    all_posts.to_csv("fb_scraper_{}.csv".format(nowtime))
    print("{} ## File written with timestamp".format(nowtime))

2022-2-2_20-29-35 ## File written with timestamp


In [7]:
all_posts.to_csv("fb_srcaper1.csv")

In [5]:
start_url

'https://m.facebook.com/groups/373920943948661?bac=MTYzMDk3NTk5Njo1MDgyMTUyMzM4NTI1NjQ6NTA4MjE1MjMzODUyNTY0LDAsMzg6MjA6S3c9PQ%3D%3D&multi_permalinks'

In [11]:
with open("test.csv", 'w+', encoding="utf-8") as f:
    for each in results[:4]:
        line = [str(i) for i in list(each)]
        line = ",".join(line)
        f.write(line)


In [13]:
list(results[0].keys())

['post_id',
 'text',
 'post_text',
 'shared_text',
 'time',
 'timestamp',
 'image',
 'image_lowquality',
 'images',
 'images_description',
 'images_lowquality',
 'images_lowquality_description',
 'video',
 'video_duration_seconds',
 'video_height',
 'video_id',
 'video_quality',
 'video_size_MB',
 'video_thumbnail',
 'video_watches',
 'video_width',
 'likes',
 'comments',
 'shares',
 'post_url',
 'link',
 'links',
 'user_id',
 'username',
 'user_url',
 'is_live',
 'factcheck',
 'shared_post_id',
 'shared_time',
 'shared_user_id',
 'shared_username',
 'shared_post_url',
 'available',
 'comments_full',
 'reactors',
 'w3_fb_url',
 'reactions',
 'reaction_count',
 'with',
 'page_id',
 'sharers',
 'image_id',
 'image_ids',
 'was_live',
 'header']